## 담당자별 엑셀 파일 하나로 합치기

In [ ]:
# 표(엑셀 데이터를 표 형태로) 다루는 라이브러리
import pandas as pd
# 파일·폴더 경로를 편하게 다루는 도구 (문자열보다 안전/명확)
from pathlib import Path

In [ ]:
# 엑셀 파일들이 들어있는 폴더 경로를 만든다. (← 네 컴퓨터의 실제 폴더로 바꿔 쓰기)
src_dir = Path(r"C:/Users/python2025")
# 합칠 파일들의 경로 리스트를 만든다: test_seoulwest_1.xlsx ~ test_seoulwest_5.xlsx
files = [src_dir / f"test_seoulwest_{i}.xlsx" for i in range(1, 6)]

In [ ]:
# 각 파일에서 읽은 표(DataFrame)를 잠시 담아둘 리스트
dfs = []
for f in files:  # 파일을 하나씩 꺼내면서
    # 엑셀을 읽는다.
    # header=4  → 엑셀 화면 기준 '5행'을 컬럼 제목(헤더)로 쓰겠다는 뜻(파이썬은 0부터 세니까)
    # usecols="A:F" → A~F 열만 읽는다(부점, 팀, 기업체명, 직/위재, 담당자, 내용)
    # engine="openpyxl" → xlsx 읽기 엔진 지정(호환성 좋음)
    df = pd.read_excel(f, header=4, usecols="A:F", engine="openpyxl")

    # 읽은 표를 리스트에 모아둔다(나중에 모두 합치려고)
    dfs.append(df)

In [ ]:
# 모아둔 여러 표를 '세로로' 이어붙여 하나의 큰 표로 만든다.
# ignore_index=True → 합친 뒤 행 번호(인덱스)를 0,1,2…로 새로 매긴다.
raw = pd.concat(dfs, ignore_index=True)

In [ ]:
# 확인용
print(raw.head(20))

In [ ]:
# 같은 실무 행을 구분해주는 '키' 컬럼들을 정한다.
KEYS = ["부점", "팀", "기업체명", "직/위재", "담당자"]

In [ ]:
# ‘내용’ 값 정리 함수: "", "none", "nan", "-" 같은 표시들은 전부 '빈값'으로 처리
def _clean_content(s):
    s = str(s).strip().lower()
    return "" if s in {"", "none", "nan", "-"} else s

In [ ]:
# ‘내용’ 칼럼에 적용
raw["내용"] = raw["내용"].map(_clean_content)

In [ ]:
# 확인용
print(raw.head(20))

In [ ]:
# 여러 파일에서 모인 ‘내용’ 중에서 '의미 있는 값' 하나를 고르는 함수
def pick_nonempty(series: pd.Series) -> str:
    # 빈 문자열("")이 아닌 값들만 모은다
    nonempties = [v for v in series if isinstance(v, str) and v != ""]
    if not nonempties:
        return ""              # 전부 빈값이면 빈문자(안전망)
    return max(nonempties, key=len)  # 가장 긴(정보가 많은) 값을 선택

In [ ]:
# 핵심 병합 단계:
# 1) 같은 키(부점·팀·기업체명·직/위재·담당자)끼리 묶고(groupby)
# 2) 그 묶음 안에서 ‘내용’을 pick_nonempty로 하나만 뽑아낸다(agg)
# as_index=False → 결과를 일반 표 형태로 유지, sort=False → 원래 순서 최대한 유지
merged = (
    raw
    .groupby(KEYS, as_index=False, sort=False)
    .agg(내용=("내용", pick_nonempty))
)

In [ ]:
# 확인용
print(merged.head(20))

In [ ]:
# 결과를 저장할 엑셀 파일 경로(같은 폴더 안에 저장)
out_path = src_dir / "merged_seoulwest_filled.xlsx"
# 엑셀로 저장: 시트 이름은 '합본', index=False → 왼쪽 인덱스 열은 저장하지 않음
merged.to_excel(out_path, sheet_name="합본", index=False)

print(f"완료! {len(merged)}행 병합 → {out_path}")